In [3]:
import re
import json
import requests

API_URL = "https://www.air-port-codes.com/api/v1/multi"
API_KEY = "d4ad68a761"


IATA_FALLBACK = {
    "Madrid": "MAD",
    "Frankfurt": "FRA",
    "Quito": "UIO",
    "Berlin": "BER",
    "Barcelona": "BCN",
    "Roma": "FCO",
    "Sevilla": "SVQ"
}


AEROLINEAS = ["iberia", "lufthansa", "aireuropa", "latam", "avianca"]

CANTIDADES_PALABRA = {
    "un": 1, "uno": 1, "dos": 2, "tres": 3, "cuatro": 4,
    "cinco": 5, "seis": 6, "siete": 7, "ocho": 8,
    "nueve": 9, "diez": 10
}

MESES = [
    "enero","febrero","marzo","abril","mayo","junio",
    "julio","agosto","septiembre","octubre","noviembre","diciembre"
]

MESES_NUM = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
}

def limpiar_ciudad(ciudad):
    if not ciudad:
        return None

    ciudad = ciudad.lower()

    for mes in MESES:
        ciudad = ciudad.replace(f" en {mes}", "")

    ciudad = re.sub(r'\b(en|para|el|la|los|las)\b', '', ciudad)
    ciudad = re.sub(r'\s+', ' ', ciudad)

    return ciudad.strip().title()


def obtener_iata_api(ciudad):
    if not ciudad:
        return "N/A"

    headers = {
        "Content-Type": "application/json",
        "APC-Auth": API_KEY
    }

    payload = {"cities": [ciudad]}

    try:
        response = requests.post(API_URL, headers=headers, json=payload, timeout=10)

        if response.status_code != 200:
            return "N/A"

        if not response.text.strip():
            return "N/A"

        data = response.json()
        ciudad_data = data.get("cities", {}).get(ciudad, [])

        if ciudad_data:
            return ciudad_data[0].get("iata", "N/A")

    except Exception:
        return "N/A"

    return "N/A"


def extraer_datos(frase):
    frase_low = frase.lower()

    datos = {
        "origen": None,
        "destino": None,
        "fecha": None,
        "cantidad": 1,
        "aerolínea": None
    }

    for aero in AEROLINEAS:
        if re.search(r'\b' + re.escape(aero) + r'\b', frase_low):
            datos["aerolínea"] = aero.capitalize()
            break

    num_match = re.search(r'\b(\d+)\b', frase_low)
    if num_match:
        datos["cantidad"] = int(num_match.group(1))
    else:
        for palabra, val in CANTIDADES_PALABRA.items():
            if re.search(r'\b' + palabra + r'\b', frase_low):
                datos["cantidad"] = val
                break

    patron = re.search(
        r'de\s+([a-záéíóúñ ]+?)\s+a\s+([a-záéíóúñ ]+?)(?:\s+en|\s+para|\s*$)',
        frase_low
    )

    if patron:
        datos["origen"] = patron.group(1).title()
        datos["destino"] = patron.group(2).title()

    # Fecha
    fecha_pattern = re.search(r'(\d{1,2})\s+de\s+(' + '|'.join(MESES) + r')', frase_low)
    if fecha_pattern:
        dia = fecha_pattern.group(1).zfill(2)
        mes = MESES_NUM[fecha_pattern.group(2)]
        datos["fecha"] = f"{dia}-{mes}-2021"
    else:
        mes_pattern = re.search(r'\b(' + '|'.join(MESES) + r')\b', frase_low)
        if mes_pattern:
            mes = MESES_NUM[mes_pattern.group(1)]
            datos["fecha"] = f"01-{mes}-2021"
        else:
            datos["fecha"] = "01-01-2021"

    return datos

def generar_json_final(datos):
    origen = limpiar_ciudad(datos.get("origen"))
    destino = limpiar_ciudad(datos.get("destino"))

    iata_from = obtener_iata_api(origen)
    iata_to = obtener_iata_api(destino)

    if iata_from == "N/A":
        iata_from = IATA_FALLBACK.get(origen, "N/A")
    if iata_to == "N/A":
        iata_to = IATA_FALLBACK.get(destino, "N/A")

    return {
        "Origen": origen,
        "Ciudad Destino": destino,
        "Nombre Ciudad IATA From": iata_from,
        "Nombre Ciudad IATA To": iata_to,
        "Fecha": datos.get("fecha"),
        "Pax": datos.get("cantidad")
    }


def asistent():
    print("Hola, bienvenido a 'TravelBot Salesiana'. ¿Cómo te puedo ayudar?")
    while True:
        texto = input(">> ").strip()

        if texto.lower() == "salir":
            print("Gracias por usar el sistema. ¡Hasta pronto!")
            break

        datos = extraer_datos(texto)

        print(
            f"\nPerfecto, comienzo la búsqueda de tu viaje a {datos['destino']} "
            f"desde {datos['origen']} para el {datos['fecha']} "
            f"con {datos['aerolínea']}.\n"
        )

        json_final = generar_json_final(datos)
        print("JSON Generado para el sistema de reservas:")
        print(json.dumps(json_final, indent=4, ensure_ascii=False))
        print("-" * 40)


In [ ]:
asistent()

Hola, bienvenido a 'TravelBot Salesiana'. ¿Cómo te puedo ayudar?
>> Quiero 2 billetes de Madrid a Frankfurt en Septiembre

Perfecto, comienzo la búsqueda de tu viaje a Frankfurt desde Madrid para el 01-09-2021 con None.

JSON Generado para el sistema de reservas:
{
    "Origen": "Madrid",
    "Ciudad Destino": "Frankfurt",
    "Nombre Ciudad IATA From": "MAD",
    "Nombre Ciudad IATA To": "FRA",
    "Fecha": "01-09-2021",
    "Pax": 2
}
----------------------------------------
>> Madrid

Perfecto, comienzo la búsqueda de tu viaje a None desde None para el 01-01-2021 con None.

JSON Generado para el sistema de reservas:
{
    "Origen": null,
    "Ciudad Destino": null,
    "Nombre Ciudad IATA From": "N/A",
    "Nombre Ciudad IATA To": "N/A",
    "Fecha": "01-01-2021",
    "Pax": 1
}
----------------------------------------
>>  Quiero dos billetes desde Quito a Sevilla

Perfecto, comienzo la búsqueda de tu viaje a Sevilla desde Quito para el 01-01-2021 con None.

JSON Generado para el si